In [1]:
import sys
import matplotlib
import math
import pyproj
from descartes import PolygonPatch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.colorbar import ColorbarBase
from mpl_toolkits.axes_grid.inset_locator import inset_axes
import sys
import os
import zipfile
import subprocess
import json
import shelve
import fiona
import urllib.request
import shutil

/Users/cyzhu/anaconda3/envs/default/lib/python3.7/site-packages/ipykernel_launcher.py:11: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  # This is added back by InteractiveShellApp.init_path()


In [2]:
class GeoProj(object):
    def __init__(self, input="epsg:4326", output="epsg:3857"):
        self._outProj = pyproj.Proj(init=output)
        self._inProj = pyproj.Proj(init=input)

    def transform(self, source):
        if isinstance(source, dict):
            return {'type': source['type'],
                    'coordinates': self.transform(source['coordinates'])}
        ans = []
        if any(isinstance(el, list) for el in source) or any(isinstance(el, tuple) for el in source):
            for el in source:
                ans.append(self.transform(el))
        else:
            ans = pyproj.transform(self._inProj, self._outProj, source[0], source[1])
        return ans
geo_proj = GeoProj()

In [3]:

cache = False
if not cache:
    try:
        shutil.rmtree('./data/') 
        
    except FileNotFoundError:
        print('No such file or directory.')

In [4]:
try:
    print('Creating data folder')
    os.makedirs('./data/')
except IOError:
    print('Folder already exists')
    pass

zip_file = "./data/zoning_wgs84.zip"
if not os.path.exists(zip_file):
    url = "http://opendata.toronto.ca/gcc/zoning_wgs84.zip"
    print('Downloading compressed shp file from data soure: \n', url)
    urllib.request.urlretrieve(url, './data/zoning_wgs84.zip')
shp_file = "./data/zoning_wgs84/ZONING_ZONE_CATAGORIES_WGS84.shp"
if not os.path.exists(shp_file):
    print('Extracting shp file')
    with zipfile.ZipFile(zip_file) as zf:
        zf.extractall('./data')

Creating data folder
 http://opendata.toronto.ca/gcc/zoning_wgs84.zip
Extracting shp file


In [9]:
matplotlib.use('ps')

In [5]:
matplotlib.get_backend()

'module://ipykernel.pylab.backend_inline'

In [19]:
shape = fiona.open("./data/ZONING_ZONE_CATAGORIES_WGS84.shp")
fig = plt.figure(figsize=(15,15))
x1, y1 = geo_proj.transform([-79.63, 43.58])
x2, y2 = geo_proj.transform([-79.11, 43.85])

ax = fig.add_subplot(1, 1, 1)
ax.set_xlim(x1, x2)
ax.set_ylim(y1, y2)


fc = "#ffffff"
ec = "#ff0000"
z = 100

while True:
    try:
        i = next(shape)
        poly = i['geometry']
        poly = geo_proj.transform(poly)     
        ax.add_patch(PolygonPatch(poly, fc=fc, ec=ec, zorder=z))
    except:
        print('Finish')
        break


/Users/cyzhu/anaconda3/envs/default/lib/python3.7/site-packages/ipykernel_launcher.py:17: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.


Finish


In [20]:
fig.savefig('output.png')